In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, fbeta_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings("ignore")

def load_image(path, label, image_size=(224, 224)):
    try:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, image_size)
        img = img.astype(np.float32) / 255.0
        return img.flatten(), label
    except Exception as e:
        print(f"Error loading image {path}: {e}")
        return None

def load_dataset(folders, image_size=(224, 224)):
    image_paths, labels = [], []

    for folder in folders:
        label = os.path.basename(folder)
        for filename in os.listdir(folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(folder, filename))
                labels.append(label)

    images, labels_final = [], []

    for path, label in zip(image_paths, labels):
        result = load_image(path, label, image_size)
        if result is not None:
            img, lbl = result
            images.append(img)
            labels_final.append(lbl)

    X = np.array(images, dtype=np.float32)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels_final)

    class_names = label_encoder.classes_.tolist()

    return X, y, class_names

def cross_val_with_pca(X, y, param_grid, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    best_score = 0
    best_params = None

    for C in param_grid['C']:
        for kernel in param_grid['kernel']:
            for degree in param_grid['degree']:
                for gamma in param_grid['gamma']:
                    
                    current_params = {'C': C, 'kernel': kernel, 'degree': degree, 'gamma': gamma}
                    fold_fbeta = []

                    for train_idx, val_idx in skf.split(X, y):

                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        pca = PCA()
                        X_train_pca = pca.fit_transform(X_train)

                        X_val_pca = pca.transform(X_val)

                        model = SVC(**current_params)
                        model.fit(X_train_pca, y_train)

                        y_pred = model.predict(X_val_pca)
                        fbeta = fbeta_score(y_val, y_pred, beta=0.5)
                        fold_fbeta.append(fbeta)

                    mean_fbeta = np.mean(fold_fbeta)

                    if mean_fbeta > best_score:
                        best_score = mean_fbeta
                        best_params = current_params.copy()

    return best_params, best_score



In [ ]:

folders = [
    "/kaggle/input/reversal2/bearish/bearish",
    "/kaggle/input/reversal2/bullish/bullish",
]

X, y, class_names = load_dataset(folders, image_size=(224, 224))

param_grid =  {
    'C': [1 , 2, 5],
    'kernel': ['poly','rbf'],
    'degree': [1,2,3],
    'gamma': ['scale', 'auto', 0.01]
}

best_params, best_score = cross_val_with_pca(X, y, param_grid, n_splits=5)
print(f"\nBest Parameters: {best_params}")
print(f"\nBest F(0.5) Score: {best_score}")


Best Parameters: {'C': 1, 'kernel': 'poly', 'degree': 1, 'gamma': 'auto'}

Best F(0.5) Score: 0.962153738927473


In [ ]:
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.decomposition import PCA

def train_final_model(X, y, best_params, model_filename="svm_final.pkl", pca_filename="pca_svm.pkl"):
 
    pca = PCA()
    X_pca = pca.fit_transform(X)

    pca.components_ = pca.components_.astype(np.float32)

    model = SVC(**best_params, probability=True)
    model.fit(X_pca, y)

    joblib.dump(model, model_filename, compress=3) 
    joblib.dump(pca, pca_filename, compress=3)     

    print(f"✅ Model saved as {model_filename}")
    print(f"✅ PCA saved as {pca_filename} (Compressed)")

    return model, pca


model_filename = "svm_final.pkl"
pca_filename = "pca_svm.pkl"

final_model, pca = train_final_model(X, y, best_params, model_filename, pca_filename)

✅ Model saved as svm_final.pkl
✅ PCA saved as pca_svm.pkl (Compressed)
